In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics.functional as MF
import dgl
import dgl.nn as dglnn
from dgl.data import AsNodePredDataset
from dgl.dataloading import DataLoader, NeighborSampler, MultiLayerFullNeighborSampler
from ogb.nodeproppred import DglNodePropPredDataset
import os
import numpy as np
from dgl.dataloading import (
    as_edge_prediction_sampler,
    DataLoader,
    MultiLayerFullNeighborSampler,
    negative_sampler,
    NeighborSampler,
)

In [ ]:
dataset = AsNodePredDataset(DglNodePropPredDataset('ogbn-products',root="/home/bear/workspace/singleGNN/data/dataset"))
folder_path="/home/bear/workspace/singleGNN/data/dataset/ogbn_products/split_lp"
os.makedirs(folder_path)
g = dataset[0]

Products数据集
nodeNUM:2449029
edgeNUM:123718152
feat:100
class:47
trainNUM:196615
valNUM:39323
testNUM:2213091

In [ ]:
import random
edgeNUM = 1000000
neg_num = 1000
sampled_edge_ids = random.sample(range(g.num_edges()), edgeNUM)
trainId=sampled_edge_ids[:int(edgeNUM * 0.3)]
TestId=sampled_edge_ids[int(edgeNUM * 0.3):int(edgeNUM * 0.9)]
ValId=sampled_edge_ids[int(edgeNUM * 0.9):]


neg_train_sampler = dgl.dataloading.negative_sampler.Uniform(1)
train_src, train_neg_dst = neg_train_sampler(g, torch.Tensor(trainId).to(torch.int64))
raw_train_src=torch.Tensor(g.edges()[0][trainId])
raw_train_dst=torch.Tensor(g.edges()[1][trainId])


neg_val_sampler = dgl.dataloading.negative_sampler.Uniform(neg_num)
val_src,val_neg_dst = neg_val_sampler(g, torch.Tensor(ValId).to(torch.int64))
raw_val_src=torch.Tensor(g.edges()[0][ValId])
raw_val_dst=torch.Tensor(g.edges()[1][ValId])
val_neg_dst.reshape(-1,neg_num)

neg_test_sampler = dgl.dataloading.negative_sampler.Uniform(neg_num)
test_src, test_neg_dst = neg_test_sampler(g, torch.Tensor(TestId).to(torch.int64))
raw_test_src=torch.Tensor(g.edges()[0][TestId])
raw_test_dst=torch.Tensor(g.edges()[1][TestId])
test_neg_dst.reshape(-1,neg_num)

{'train': {'source_node': tensor([ 837791,  837791,  837791,  ...,  928474, 1521923, 1257024]),
  'target_node': tensor([    102, 1074631, 1658157,  ...,  250277,  316321,  325910])},
 'valid': {'source_node': tensor([2302382, 1112061, 1660093,  ..., 1660079, 2395010,   88681]),
  'target_node': tensor([ 245742,  839129,  228003,  ...,  699000, 2485204,  849448]),
  'target_node_neg': tensor([[2415348,  412967, 2565382,  ..., 1353044, 1652394, 1627182],
          ...,
          [ 576938, 2577235, 2070601,  ...,  155330, 1643711, 1166252]])},
 'test': {'source_node': tensor([2302382, 1112061, 1660093,  ..., 1660079, 2395010,   88681]),
  'target_node': tensor([1611317,  963234,  108714,  ..., 2537726,  163858,  112969]),
  'target_node_neg': tensor([[ 114076, 2093394, 1308822,  ..., 2321062, 1852878,  936672],
          ...,
          [2587770, 2587954,   49452,  ..., 1938683, 1319512, 2029363]])}}

In [ ]:
split_lp={}
split_lp['train']={}
split_lp['train']['source_node']=raw_train_src
split_lp['train']['target_node']=raw_train_dst
split_lp['train']['target_node_neg']=raw_train_src

split_lp['valid']={}
split_lp['valid']['source_node']=raw_val_src
split_lp['valid']['target_node']=raw_val_dst
split_lp['valid']['target_node_neg']=val_neg_dst.reshape(-1,neg_num)

split_lp['test']={}
split_lp['test']['source_node']=raw_test_src
split_lp['test']['target_node']=raw_test_dst
split_lp['test']['target_node_neg']=test_neg_dst.reshape(-1,neg_num)

split_lp

In [ ]:
file_name = folder_path+"/split_dict.pt"
torch.save(split_lp, file_name)

In [ ]:
curDir = "/home/bear/workspace/singleGNN/src/predata/LP"
def load_reddit(self_loop=True):
    from dgl.data import RedditDataset
    data = RedditDataset(self_loop=self_loop,raw_dir=curDir+'/../../../data/dataset/')
    g = data[0]
    g.ndata['feat'] = g.ndata.pop('feat')
    g.ndata['label'] = g.ndata.pop('label')
    train_idx = []
    val_idx = []
    test_idx = []
    for index in range(len(g.ndata['train_mask'])):
        if g.ndata['train_mask'][index] == 1:
            train_idx.append(index)
    for index in range(len(g.ndata['val_mask'])):
        if g.ndata['val_mask'][index] == 1:
            val_idx.append(index)
    for index in range(len(g.ndata['test_mask'])):
        if g.ndata['test_mask'][index] == 1:
            test_idx.append(index)
    return g, data,train_idx,val_idx,test_idx

g, Testdata , train_idx , val_idx,test_idx = load_reddit()

reddit数据集
nodeNUM:232965
edgeNUM:114848857
feat:602
class:41
trainNUM:153431
valNUM:23831
testNUM:55703

In [ ]:
import random
edgeNUM = 1000000
neg_num = 1000
sampled_edge_ids = random.sample(range(g.num_edges()), edgeNUM)
trainId=sampled_edge_ids[:int(edgeNUM * 0.3)]
TestId=sampled_edge_ids[int(edgeNUM * 0.3):int(edgeNUM * 0.9)]
ValId=sampled_edge_ids[int(edgeNUM * 0.9):]


neg_train_sampler = dgl.dataloading.negative_sampler.Uniform(1)
train_src, train_neg_dst = neg_train_sampler(g, torch.Tensor(trainId).to(torch.int64))
raw_train_src=torch.Tensor(g.edges()[0][trainId])
raw_train_dst=torch.Tensor(g.edges()[1][trainId])


neg_val_sampler = dgl.dataloading.negative_sampler.Uniform(neg_num)
val_src,val_neg_dst = neg_val_sampler(g, torch.Tensor(ValId).to(torch.int64))
raw_val_src=torch.Tensor(g.edges()[0][ValId])
raw_val_dst=torch.Tensor(g.edges()[1][ValId])
val_neg_dst.reshape(-1,neg_num)

neg_test_sampler = dgl.dataloading.negative_sampler.Uniform(neg_num)
test_src, test_neg_dst = neg_test_sampler(g, torch.Tensor(TestId).to(torch.int64))
raw_test_src=torch.Tensor(g.edges()[0][TestId])
raw_test_dst=torch.Tensor(g.edges()[1][TestId])
test_neg_dst.reshape(-1,neg_num)

In [ ]:
split_lp={}
split_lp['train']={}
split_lp['train']['source_node']=raw_train_src
split_lp['train']['target_node']=raw_train_dst
split_lp['train']['target_node_neg']=raw_train_src

split_lp['valid']={}
split_lp['valid']['source_node']=raw_val_src
split_lp['valid']['target_node']=raw_val_dst
split_lp['valid']['target_node_neg']=val_neg_dst.reshape(-1,neg_num)

split_lp['test']={}
split_lp['test']['source_node']=raw_test_src
split_lp['test']['target_node']=raw_test_dst
split_lp['test']['target_node_neg']=test_neg_dst.reshape(-1,neg_num)

split_lp

In [ ]:
folder_path=curDir+'/../../../data/dataset/split_lp'
os.makedirs(folder_path)
file_name = folder_path+"/split_dict.pt"
torch.save(split_lp, file_name)